In [1]:
import pandas as pd
import re
import pickle
import os.path
from catboost import CatBoostClassifier
# Функция выделения даты
def get_data(s):
    dat = re.search(r'\d\d\.\d\d\.\d\d\d\d', s)
    if dat is not None:
        dat = dat.group(0)
    return dat


# Функция выделения номера договора
def get_num(s):
    return re.sub(r'\d\d\.\d\d\.\d\d\d\d', '', s)


# Функция выделения названия организации
def get_org(s):
    org = re.split(r'\(', s)
    return org[1][0:-1]


# Функция выделения автора
def get_name(s):
    org = re.split(r'\(', s)
    return org[0]


def preprocess(name):
    intro_df = None
    try:
        intro_df = pd.read_html(name)
        intro_df = intro_df[1]
    finally:
        intro_df = pd.read_csv(name)

    intro_df['Дата'] = intro_df['Исх. №Дата'].map(get_data)
    intro_df['Исх. №'] = intro_df['Исх. №Дата'].map(get_num)

    # Удаляем исходный столбик за ненадобностью
    intro_df.drop('Исх. №Дата', axis=1, inplace=True)

    # Применение вышеуказанных функций
    intro_df['Организация'] = intro_df['Автор'].map(get_org)
    intro_df['Автор'] = intro_df['Автор'].map(get_name)

    # Выделение вышеописаных атрибутов в отдельный DataFrame
    resulted_df = intro_df[['Адресат', 'Автор', 'Краткоесодержание', 'Примечание', 'Организация']]

    # Приведем слова к общему виду, поставив все слова в именительный падеж и удалив знаки препинания

    import pymorphy2 as mrf
    morph = mrf.MorphAnalyzer()

    # Функция преобразующая все слова в тексте в именительный падеж
    def txt_normolize(s):
        temp = s.split(' ')
        res = [morph.parse(temp[i])[0].normal_form.rstrip('-,') for i in range(len(temp))]
        res = [x for x in res if x]
        return res

    resulted_df['Краткоесодержание'] = resulted_df['Краткоесодержание'].map(txt_normolize)

    return resulted_df


def feature_eng(df):
    reeng_feach = pd.DataFrame()

    stat = {}
    for i in df['Автор']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    autor = []
    for i in df['Автор']:
        autor.append(stat.index(i))
    reeng_feach['Автор'] = autor

    stat = {}
    for i in df['Адресат']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    adres = []
    for i in df['Адресат']:
        adres.append(stat.index(i))
    reeng_feach['Адресат'] = adres

    stat = {}
    for i in df['Организация']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    org = []
    for i in df['Организация']:
        org.append(stat.index(i))
    reeng_feach['Организация'] = org

    dic = {}
    for i in df['Краткоесодержание']:
        for j in i:
            if j not in dic:
                dic[j] = 1
            else:
                dic[j] += 1

    def get_pop(m):
        t = 0
        temp = []
        for i in m:
            if dic[i] > t:
                temp.append(i)
                t = dic[i]
                if len(temp) >= 4:
                    temp = temp[1:]
        while len(temp) < 3:
            temp.append('')
        return temp

    sod = []
    for i in df['Краткоесодержание']:
        sod.append(get_pop(i))
    key1 = []
    key2 = []
    key3 = []
    for i in sod:
        key1.append(i[0])
        key2.append(i[1])
        key3.append(i[2])
    reeng_feach['Keyword1'] = key1
    reeng_feach['Keyword2'] = key2
    reeng_feach['Keyword3'] = key3

    stat = {}
    for i in reeng_feach['Keyword1']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword1']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword1'] = soderj

    stat = {}
    for i in reeng_feach['Keyword2']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword2']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword2'] = soderj

    stat = {}
    for i in reeng_feach['Keyword3']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword3']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword3'] = soderj
    return reeng_feach

def feature_eng_out(df):
    reeng_feach = pd.DataFrame()

    stat = {}
    for i in df['Автор']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    autor = []
    for i in df['Автор']:
        autor.append(stat.index(i))
    reeng_feach['Автор'] = autor

    stat = {}
    for i in df['Организация']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    org = []
    for i in df['Организация']:
        org.append(stat.index(i))
    reeng_feach['Организация'] = org

    dic = {}
    for i in df['Краткоесодержание']:
        for j in i:
            if j not in dic:
                dic[j] = 1
            else:
                dic[j] += 1

    def get_pop(m):
        t = 0
        temp = []
        for i in m:
            if dic[i] > t:
                temp.append(i)
                t = dic[i]
                if len(temp) >= 4:
                    temp = temp[1:]
        while len(temp) < 3:
            temp.append('')
        return temp

    sod = []
    for i in df['Краткоесодержание']:
        sod.append(get_pop(i))
    key1 = []
    key2 = []
    key3 = []
    for i in sod:
        key1.append(i[0])
        key2.append(i[1])
        key3.append(i[2])
    reeng_feach['Keyword1'] = key1
    reeng_feach['Keyword2'] = key2
    reeng_feach['Keyword3'] = key3

    stat = {}
    for i in reeng_feach['Keyword1']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword1']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword1'] = soderj

    stat = {}
    for i in reeng_feach['Keyword2']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword2']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword2'] = soderj

    stat = {}
    for i in reeng_feach['Keyword3']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword3']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword3'] = soderj
    return reeng_feach


def get_mails(s, df):
    # if s is not str:
    return df[df['Адресат'] == s] 

def relearn():
    CBC = CatBoostClassifier()
    feach = preprocess('Выгрузка 9 апреля.xls')
    feach = feature_eng(feach)
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(feach.drop('Адресат',axis=1), feach['Адресат'], test_size=0.15)
    CBC.fit(x_train,y_train)
    return CBC

In [2]:
imported = pd.read_html('Выгрузка 9 апреля.xls')
imported = imported[1]

In [4]:
if os.path.isfile('model.pic'):
    with open('model.pic','rb') as f:
        model = pickle.load(f)
else:
    model = relearn()
    
with open('names.pic','rb') as f:
    names = pickle.load(f)    

pd.options.display.max_rows = 100
put = ['']
while put[0] != 'exit':
    put = input()

    put = re.split(' ', put, maxsplit=1)

    if put[0] == 'find_letters':
        print(get_mails(put[1], imported))
    elif put[0] == 'get_recipient':
        print('Введите Фамилию и инициалы')
        name = str(input())
        print('Введите название организации')
        org = str(input())
        print('Введите краткое содержание письма')
        let = str(input())
        ldf = {
            'Автор' : [name],
            'Организация' : [org],
            'Краткоесодержание' : [let]
        }
        
        ldf = pd.DataFrame(ldf)
        
        ldf = feature_eng_out(ldf)
        pred = model.predict(ldf)
        print('Предположительно адресат даного письма:')
        print(names[int(pred)])
    elif put[0] == 'help':
        print('(функция) [аргументы] - пишется функция и через пробел аргументы, если они требуются \n' +
              'find_letters - поиск всех писем для указанного адресанта, адресат указывается в качестве аргумента, фимлия пишется с большой буквы, \n инициалы заглавными буквами через точку, например: Иванов И.И. \n' +
              'get_recipient - предсказание адресата по необходимой информации из письма \n' +
              'help - показать подсказку по имеющемся функциям \n'+
              'exit - выход из программы'
              )
    else: print('Команда {} не найдена'.format(put[0]))
        

help
(функция) [аргументы] - пишется функция и через пробел аргументы, если они требуются 
find_letters - поиск всех писем для указанного адресатаб адресат указывается в качестве арггумета, фимлия пишется с большой буквы, 
 инициалы заглавными буквами через точку, например: Иванов И.И. 
get_recipient - предсказание адресата по необходимой информации из письма 
help - показать подсказку по имеющемся функциям



KeyboardInterrupt: 